In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str
    link: List[str]
    text: str
    
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library 
        """
        self.url = url
        response = requests.get(url)
        self.boday = response.content
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())
print(ed.links)

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Career/Jobs pages.\n"
link_system_prompt += "You should response in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Career/Jobs pages.
You should response in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of the links on the website of {website.url} - "
    user_prompt += "Please decide which of these are relevant web links for a brochure about the company, response with the full https URL in JSON format: \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of the links on the website of https://edwarddonner.com - Please decide which of these are relevant web links for a brochure about the company, response with the full https URL in JSON format: Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://e

In [12]:

def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format = {"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)


In [13]:
get_links("https://anthropic.com")

{'links': [{'type': 'company page',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'economic futures page',
   'url': 'https://www.anthropic.com/economic-futures'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'}]}

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    
    links = get_links(url) 
    print("Found links:", links)

    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


In [16]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'about page', 'url': 'https://www.anthropic.com/about'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Research
Economic Futures
Commitments
Initiatives
Transparency
Responsible Scaling Policy
Trust center
Security and compliance
Learn
Learn
Anthropic Academy
Engineering at Anthropic
Developer docs
Company
About
Careers
Events
News
Try Claude
Try Claude
Try Claude
Learn more about Claude
Overview
Meet Claude
Pricing
Products
Claude
Claude Console
Claude Code
Claude Developer Platform
Models
Opus
Sonnet
Haiku
Log in to Claude
Log in to Claude
Log in to Claude
EN
This is some text inside of a div block.
Log in to Claude
Log in to Claude
Log in to Claude
Download app
Download app
Download app
Research
Economic Futures
Commitments
Initiatives
Transparency
Responsibl

In [18]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company:\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]  # Truncate if more than 20,000 characters
    return user_prompt


In [21]:
print(get_brochure_user_prompt("Anthropic", "https://anthropic.com"))

Found links: {'links': [{'type': 'main page', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'economic futures page', 'url': 'https://www.anthropic.com/economic-futures'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}
You are looking at a company called: Anthropic
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company:
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Research
Economic Futures
Commitments
Initiatives
Transparency
Responsible Scaling Policy
Trust center
Security and compliance


In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'economic futures page', 'url': 'https://www.anthropic.com/economic-futures'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Brochure

## About Us
**Anthropic** is a public benefit corporation based in San Francisco, dedicated to advancing the field of artificial intelligence (AI) with emphasis on safety, reliability, and human benefit. We build and research AI systems, including our advanced models like Claude, Opus, Sonnet, and Haiku. Our commitment to AI safety aligns with our responsibility to mitigate the risks of emerging technologies while harnessing their potential for societal good.

## Our Mission
At Anthropic, we believe AI can vastly impact the world. Our mission is to develop AI systems that people can trust, enhancing the benefits while minimizing the threats. We undertake rigorous research into AI safety, ensuring our technology acts as a robustly positive force for humanity.

## Core Values
- **Act for the Global Good**: We make decisions maximizing long-term benefits for humanity.
- **Hold Light and Shade**: We analyze both the potential benefits and risks associated with AI.
- **Be Good to Our Users**: We prioritize generosity and kindness towards everyone affected by our technology.
- **Ignite a Race to the Top on Safety**: We strive for industry-wide excellence in AI safety.
- **Do the Simple Thing That Works**: We focus on practical solutions rather than overly complex ones.
- **Be Helpful, Honest, and Harmless**: We maintain a high-trust, collaborative environment.
- **Put the Mission First**: Our mission guides our actions and decisions, reinforcing collective responsibility.

## Company Culture
Our team embodies a diverse range of expertise—from machine learning and engineering to public policy and operational excellence. We foster an inclusive, supportive workplace that values transparency, collaboration, and mutual respect. Each team member plays a crucial role in shaping the future of safe AI through their unique contributions.

## Products and Services
Our flagship product, **Claude**, is at the forefront of AI innovation, serving businesses and civil society with robust tools. Other offerings include:
- Claude Console
- Claude Code
- Claude Developer Platform

Through our **Anthropic Academy**, we also provide training and resources for developers and organizations interested in utilizing our AI technology effectively.

## Economic Contributions
**Anthropic Economic Index**: We conduct foundational research to understand AI's evolving impact on the labor market. Our studies aim to provide insights that can guide policymakers and society toward an informed future.

## Careers at Anthropic
We are always seeking passionate individuals to join our mission of creating safe AI systems. Positions are available across various domains, including research, engineering, policy, and operations. 

### Employee Benefits:
- **Health & Wellness**: Comprehensive health coverage, mental health support, fertility benefits, and generous parental leave.
- **Compensation**: Competitive salary and equity packages, retirement plans, and optional equity donation matching.
- **Work-Life Balance**: Flexible paid time off, wellness stipends, and home office support.

### Join Us!
If you believe in making a positive impact through AI, consider applying to become part of the Anthropic team. Explore open roles on our [Careers Page](#).

## Contact Us
For more information or to learn about our products, please visit our website at [Anthropic.com](#).

---
**Anthropic PBC** © 2024. All rights reserved.

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.anthropic.com/'}, {'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'economic futures page', 'url': 'https://www.anthropic.com/economic-futures'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic: Building AI for Humanity's Future

Welcome to Anthropic, a pioneering public benefit corporation dedicated to creating safe and reliable AI systems. Our mission centers on ensuring that the benefits of artificial intelligence are realized and its risks mitigated. Here's a snapshot of who we are, what we do, and how you can be a part of our journey.

## Our Purpose

At Anthropic, we believe in the transformative potential of AI and its profound impact on society. By developing advanced AI technologies, including tools like **Claude**, we strive to create systems that you can rely on for their interpretability, safety, and usability. Our commitment extends to conducting research that explores both the opportunities and challenges that AI presents.

### Our Core Values and Culture

At Anthropic, our values guide every decision we make, fostering a culture of responsibility, collaboration, and innovation:

1. **Act for the Global Good**: We prioritize long-term positive outcomes for humanity, navigating the technological revolution with safety and accountability.
2. **Hold Light and Shade**: We recognize the dual potential of AI, balancing risks with benefits through informed action.
3. **Be Good to Our Users**: We treat everyone—customers, team members, and impacted communities—with generosity and kindness.
4. **Ignite a Race to the Top on Safety**: We aim to set industry standards for AI safety, encouraging others to do the same.
5. **Do the Simple Thing That Works**: We focus on empirical solutions and prioritize impact over complexity.
6. **Be Helpful, Honest, and Harmless**: Our organization thrives on trust, open communication, and thoughtfulness in our actions.
7. **Put the Mission First**: Our mission is the foundation of our work ethos, driving swift and unified actions toward our goals.

## Our Products

Anthropic develops multiple AI products designed to serve various sectors, including:

- **Claude**: An advanced AI assistant designed for safe and reliable interaction.
- **Claude Console and Claude Code**: Tools that enhance user engagement and functionality.
- **Claude Developer Platform**: Offers robust development resources for businesses across diverse industries.

## Organizational Structure

Our interdisciplinary team comprises researchers, engineers, policy experts, and operational leaders. Together, we foster a collaborative environment that values diverse backgrounds, from scientific disciplines to public policy, ensuring we remain at the forefront of AI safety research.

## Join the Anthropic Team

Looking to build a career in AI that prioritizes safety and responsibility? Anthropic is seeking talented individuals who share our vision. We have a range of open roles for those passionate about making a real-world impact through technology.

### Why Work at Anthropic?

- **Innovative Environment**: Engage with cutting-edge AI technologies and participate in meaningful research.
- **Mission-Driven Culture**: Work alongside individuals committed to contributing positively to humanity through responsible AI.
- **Growth Opportunities**: Benefit from a learning culture, with access to Anthropic Academy for skill enhancement and professional development.

## Connect with Us

For more information on our products, values, or career opportunities, visit our website or reach out to us directly. Join us at Anthropic to be part of the crucial mission of safe AI development and its benefits to society!

---

Anthropic PBC  
Headquartered in San Francisco  
[Explore Careers](#) | [Learn About Our Products](#) | [Follow Our Research](#)  

*Together, let's shape the future of AI for the betterment of all.*

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face
**The AI Community Building the Future**  
At Hugging Face, we provide an inclusive platform for the machine learning community to collaborate on models, datasets, and applications. With over **1 million models** and **250,000 datasets** at your fingertips, we are dedicated to democratizing access to AI.

---

### Company Mission
Hugging Face aims to **democratize good machine learning, one commit at a time**. We believe in open-source collaboration, inviting talent from around the globe to contribute to the future of AI.

---

### Community Engagement
Join a thriving community of over **50,000 organizations** including industry giants like **Google**, **Microsoft**, and **Amazon**. Collaborate on projects, share your work, and interact with others on our platform. Our forums and blogs are filled with insightful articles, ensuring you stay updated on the latest in AI trends.

---

### AI Solutions
- **Models & Datasets**: Access cutting-edge models and diverse datasets to fuel your AI projects.
- **Spaces**: Create, deploy, and share your applications with ease.
- **Enterprise Solutions**: Designed with advanced security and support, our enterprise offerings include compute solutions starting at **$20/user/month**.

---

### Careers at Hugging Face
Explore a world of opportunities at Hugging Face. We are committed to fostering a dynamic and inclusive workplace where talent thrives. Current openings span various domains from engineering to product management. 

If you are passionate about AI and want to make an impact, consider becoming part of our team. **Join us in shaping the future!**

---

### Company Culture
At Hugging Face, we embrace diversity and innovation. Our team works collaboratively and values transparency and open communication. We encourage employees to explore their ideas, share their insights, and grow both professionally and personally.

---

### Engage with Us
Discover more about our projects and initiatives:
- **Website**: [huggingface.co](https://huggingface.co/)
- **GitHub**: [Hugging Face on GitHub](https://github.com/huggingface)
- **Social Media**: Follow us on [Twitter](https://twitter.com/huggingface) and [LinkedIn](https://linkedin.com/company/huggingface).

Join us as we build the future of AI together!

--- 

*For inquiries and additional information, feel free to reach out through our website.*  

--- 

Feel free to adjust this brochure to better fit your branding and communication goals!